# Regressão Logística

## Importações

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

## Leitura de Dados

In [2]:
treinamento = pd.read_csv('diabetes_train.csv')

In [3]:
teste = pd.read_csv('diabetes_test.csv')

## Pré-processamento

In [4]:
class Encoders:
    def __init__(self):
        self.encoders = {}

    def add_encoder(self, feature, encoder):
        self.encoders[feature] = encoder

    def get_encoder(self, feature):
        try:
            return self.encoders[feature]
        except KeyError:
            return None

In [5]:
encoders = Encoders()

In [6]:
def cria_encoder(feature, nome, encoders):
    encoder = LabelEncoder()
    feature_codificada = encoder.fit_transform(feature)
    encoders.add_encoder(nome, encoder)
    return feature_codificada


In [7]:
def verifica_features_categoricas(dados, encoders):
    for i in range(int(dados.size/len(dados)) - 1):
        feature = dados.iloc[:, i]
        if not all(isinstance(instancia, (int, float)) for instancia in (feature.values).tolist()):
            y = encoders.get_encoder(feature.name)
            if y == None:
                dados.loc[:, feature.name] = cria_encoder((feature.values).tolist(), feature.name, encoders)
            else:
                dados.loc[:, feature.name] = y.transform((feature.values).tolist())
    
    return dados

In [8]:
treinamento = verifica_features_categoricas(treinamento, encoders)
teste = verifica_features_categoricas(teste, encoders)

In [9]:
x_treinamento, x_teste = treinamento.iloc[:, :-1].values, teste.iloc[:, :-1].values

In [10]:
y_treinamento, y_teste = treinamento.iloc[:, -1].values, teste.iloc[:, -1].values

## Algoritmo

In [11]:
class LogisticRegression:
    def __init__(self, alpha = 1, l = 1, tolerancia = 0.0001):
        self.taxa_aprendizado = alpha
        self.param_regularizacao = l
        self.tolerancia = tolerancia
        self.qtd_features = None
        self.qtd_instancias = None
        self.parametros = None

    def fit(self, x, y):
        self.qtd_instancias = len(x)
        self.qtd_features = int(x.size/len(x))
        self.parametros = np.ones(self.qtd_features + 1).reshape(-1, 1)

        matriz_design = np.column_stack([np.ones(self.qtd_instancias), x])
        y = np.array(y).reshape(-1, 1)

        erro = 1000

        while(erro > self.tolerancia):
            regressao_linear = (np.dot(matriz_design, self.parametros)).astype(np.float64)
            y_previsto = list(map(self.funcao_logistica, regressao_linear))
            y_previsto = np.array(y_previsto).reshape(-1, 1)
            matriz_transposta = matriz_design.T
            vetor_gradiente = np.dot(matriz_transposta, y_previsto - y)

            penalidade_ridge = self.parametros.copy() * self.param_regularizacao
            penalidade_ridge[0] = 0

            novos_parametros = self.parametros - (vetor_gradiente + penalidade_ridge) * (self.taxa_aprendizado/self.qtd_instancias)
            erro = np.sum(abs(self.parametros - novos_parametros))/len(self.parametros)
            self.parametros = novos_parametros
        
    def funcao_logistica(self, t):
        return 1/(1 + np.exp(-t))
    
    def prob_logistica(self, p):
        return 1 if p >= 0.5 else 0
    
    def predict(self, x):
        self.qtd_instancias = len(x)

        matriz_design = np.column_stack([np.ones(self.qtd_instancias), x])
        regressao_linear = (np.dot(matriz_design, self.parametros)).astype(np.float64)
        y_previsto = list(map(self.funcao_logistica, regressao_linear))

        return list(map(self.prob_logistica, y_previsto))

In [12]:
regressao_logistica = LogisticRegression()
regressao_logistica.fit(x_treinamento, y_treinamento)
y_resultado = regressao_logistica.predict(x_teste)

## Métricas

In [13]:
acuracia = np.sum((np.array(y_teste) == np.array(y_resultado)).astype(int))/len(y_teste) * 100

In [14]:
print(f"Acurácia: {acuracia:.2f}%")

Acurácia: 93.75%


In [15]:
matriz_confusao = np.zeros((2, 2))

for i, k in zip(y_teste, y_resultado):
    matriz_confusao[i][k] += 1

In [16]:
matriz_confusao

array([[ 74.,   7.],
       [  6., 121.]])